In [95]:
import numpy as np
from scipy.special import expit
from smote_variants import MetricTensor

class FullRankTransformer:
    """
    Transforms data into a lower dimensional full rank covariance 
    representation
    """
    def __init__(self, eps=0.002):
        """
        Constructor of the object.

        Args:
            eps (float): the tolerance
        """
        self.eps = eps
        self.mean = None
        self.transformation = None
        self.n_dim = None
    
    def fit(self, X):
        """
        Fits the transformer to the data

        Args:
            X (np.array): all vectors to fit on
        
        Returns:
            self: the fitted object
        """
        cov = np.cov(X, rowvar=False)
        eigw, eigv = np.linalg.eig(cov)
    
        eigv_f = eigv[:, eigw > self.eps]
        eigw_f = eigw[eigw > self.eps]

        self.mean = np.mean(X, axis=0)
        self.transformation = eigv_f
        self.n_dim = eigv_f.shape[1]

        return self
    
    def transform(self, X):
        """
        Transforms the data 

        Args:
            X (np.array): the data to transform
        
        Returns:
            np.array: the transformed data
        """
        return np.dot(X - self.mean, self.transformation)
    
    def inverse_transform(self, X):
        """
        Inverse transformation

        Args:
            X (np.array): the data to inverse transform
        
        Returns:
            np.array: the inverse transformed data
        """
        return np.dot(X, self.transformation.T) + self.mean

    def fit_transform(self, X):
        """
        Fits and transforms the data.

        Args:
            X (np.array): the data to fit and transform
        
        Returns:
            np.array: the transformed data
        """
        return self.fit(X).transform(X)


In [ ]:
class PDFOSKDE:
    def __init__(self, metric_learning=None, random_state=None):
        """
        Constructor of the PDFOS KDE estimation

        Args:
            metric_learning (str/None): metric learning method
            random_state (int/None/np.random.RandomState): random state
                                                            initializer
        """
        self.random_state=random_state
        self.transformer = FullRankTransformer(eps=0.02)
        self.metric_learning = metric_learning
        self.S = None
        self.X_base = None
    
    def eq_5_0(self,
                sigma,
                m # pylint: disable=invalid-name
                ):
        """
        Eq (5) in the paper

        Args:
            sigma (float): the sigma value
            m (int): the dimensionality

        Returns:
            float: the value of the equation
        """
        return sigma**(-m)/((2.0*np.pi)**(m/2))

    def eq_8_vectorized(self,
                        X,
                        sigma,
                        S_inv # pylint: disable=invalid-name
                        ):
        """
        Eq (8) in the paper

        Args:
            X (np.array): all training vectors
            sigma (float): the sigma value
            S_inv (np.array): the inverse covariance matrix

        Returns:
            np.array: the result of the equation for all pairs of vectors
        """
        m = X.shape[1] # pylint: disable=invalid-name

        tmp= (X[:,None] - X)
        tmp = np.einsum('ijk,kl,lji -> ij', tmp, S_inv, tmp.T)

        numerator_9 = (np.sqrt(2)*sigma)**(-m)*expit(-(1/(4*sigma**2))*tmp)
        numerator_5 = sigma**(-m)*expit(-(1/(2*sigma**2))*tmp)
        denominator = ((2*np.pi)**(m/2))
        eq_9 = numerator_9 / denominator
        eq_5 = numerator_5 / denominator

        return eq_9 - 2 * eq_5

    def M(self, # pylint: disable=invalid-name
            sigma,
            X,
            S_inv # pylint: disable=invalid-name
            ):
        """
        Eq (7) in the paper

        Args:
            X (np.array): all training vectors
            sigma (float): the sigma value
            S_inv (np.array): the inverse covariance matrix

        Returns:
            float: the value of the equation
        """
        m = X.shape[1] # pylint: disable=invalid-name
        total = np.sum(self.eq_8_vectorized(X, sigma, S_inv))

        term_a = total/len(X)**2
        term_b = 2.0 * self.eq_5_0(sigma, m)/len(X)
        return term_a + term_b

    def find_best_sigma(self,
                        X,
                        n_optimize,
                        S_inv # pylint: disable=invalid-name
                        ):
        """
        Find the best sigma parameter.

        Args:
            X (np.array): all training vectors
            n_optimize (int): the number of vectors used for
                                the estimation
            S_inv (np.array): the inverse covariance matrix

        Returns:
            float: the best sigma value
        """
        # finding the best sigma parameter
        best_sigma = 0
        error = np.inf
        # the dataset is reduced to make the optimization more efficient
        domain = range(len(X))
        n_to_choose = min([len(X), n_optimize])
        X_reduced = X[self.random_state.choice(domain, # pylint: disable=invalid-name
                                               n_to_choose,
                                               replace=False)]

        # we suppose that the data is normalized, thus, this search space
        # should be meaningful
        for sigma in np.logspace(-5, 2, num=20):
            err = self.M(sigma, X_reduced, S_inv)
            if err < error:
                error = err
                best_sigma = sigma
        #_logger.info("%s: best sigma found %f",
        #                self.__class__.__name__, best_sigma)

        return best_sigma

    def fit(self, X, n_optimize=100, X_ml=None, y_ml=None):
        """
        Fitting the kernel density estimator

        Args:
            X (np.array): the training vectors
            n_optimize (int): the number of random samples used
                                for optimization
            X_ml (np.array/None): potential training vectors for
                                    metric learning
            y_ml (np.array/None): potential target labels for metric
                                    learning
        
        Returns:
            obj: the fitted object
        """
        self.transformer.fit(X)
        X_trans = self.transformer.transform(X)
        S = np.cov(X_trans, rowvar=False)

        if self.metric_learning is None:
            S_inv = np.linalg.inv(S) # pylint: disable=invalid-name
            det = np.linalg.det(S)
        else:
            metrict = MetricTensor(metric_learning_method=self.metric_learning)
            S_inv = metrict.tensor(self.transformer.transform(X_ml), y_ml) # pylint: disable=invalid-name
        
        best_sigma = self.find_best_sigma(X_trans, n_optimize, S_inv)

        self.S = best_sigma * S_inv
        self.X_base = X_trans

        return self
    
    def sample(self, n_to_sample):
        """
        Draw a random sample from the fitted KDE

        Args:
            n_to_sample (int): the number of samples to generate
        
        Returns:
            np.array: the generated samples
        """
        base_indices = np.random.choice(self.X_base.shape[0], n_to_sample)
        unique_indices, unique_counts = np.unique(base_indices, return_counts=True)
        samples = []
        for idx, base_idx in enumerate(unique_indices):
            samples.append(np.random.multivariate_normal(self.X_base[base_idx],
                                                        self.S,
                                                        size=unique_counts[idx]))
        samples = np.vstack(samples)
        return self.transformer.inverse_transform(samples)
        



In [39]:
eps = 0.05

In [22]:
X = np.random.rand(10, 3)

In [74]:

X

array([[0.95515574, 0.45555325, 0.0591839 ],
       [0.49187314, 0.34902139, 0.7120596 ],
       [0.86788513, 0.36493768, 0.60733686],
       [0.8891843 , 0.66633005, 0.81471584],
       [0.83802862, 0.96883697, 0.85722615],
       [0.21420896, 0.71690079, 0.72196591],
       [0.62102805, 0.19158329, 0.0365729 ],
       [0.10456387, 0.90478402, 0.65708623],
       [0.29692751, 0.94980647, 0.17770021],
       [0.99378133, 0.25524394, 0.60822624]])

In [103]:
frt = FullRankTransformer(eps = 0.1)

In [97]:
frt.fit(X)

In [104]:
X_trans = frt.fit_transform(X)

In [105]:
X_trans

array([[ 0.46614617],
       [-0.00805462],
       [ 0.27668192],
       [ 0.02792672],
       [-0.21516258],
       [-0.43788782],
       [ 0.41228367],
       [-0.61135748],
       [-0.34381764],
       [ 0.43324165]])

In [109]:
np.cov(frt.inverse_transform(X_trans), rowvar=False)

array([[ 0.07295326, -0.06764608, -0.03615592],
       [-0.06764608,  0.06272497,  0.03352566],
       [-0.03615592,  0.03352566,  0.01791902]])

In [110]:
np.cov(X, rowvar=False)

array([[ 0.10934357, -0.04686621, -0.00160792],
       [-0.04686621,  0.08788905,  0.0283732 ],
       [-0.00160792,  0.0283732 ,  0.09726794]])

In [107]:
np.cov(X_trans, rowvar=False)

array(0.15359725)

In [32]:
cov = np.cov(X, rowvar=False)

In [33]:
cov

array([[ 0.10934357, -0.04686621, -0.00160792],
       [-0.04686621,  0.08788905,  0.0283732 ],
       [-0.00160792,  0.0283732 ,  0.09726794]])

In [34]:
eigw, eigv = np.linalg.eig(cov)

In [35]:
eigw = np.real(eigw)

In [36]:
eigv = np.real(eigv)

In [38]:
eigw

array([0.04189734, 0.15359725, 0.09900596])

In [52]:
transformation = eigv.T[eigw > eps]

In [57]:
eigv

array([[ 0.52250532,  0.68917678,  0.50201948],
       [ 0.7648756 , -0.6390407 ,  0.08119295],
       [-0.37676717, -0.34155871,  0.86103667]])

In [53]:
transformation

array([[ 0.68917678, -0.6390407 , -0.34155871],
       [ 0.50201948,  0.08119295,  0.86103667]])

In [54]:
np.dot(np.dot(eigv, np.diag(eigw)), eigv.T)

array([[ 0.10934357, -0.04686621, -0.00160792],
       [-0.04686621,  0.08788905,  0.0283732 ],
       [-0.00160792,  0.0283732 ,  0.09726794]])

In [62]:
eigv_f = eigv[:, eigw > eps]
eigw_f = eigw[eigw > eps]

In [63]:
np.dot(np.dot(eigv_f, np.diag(eigw_f)), eigv_f.T)

array([[ 0.0979051 , -0.06361055,  0.00664012],
       [-0.06361055,  0.06337765,  0.04044718],
       [ 0.00664012,  0.04044718,  0.09132047]])

In [84]:
X_trans = np.dot(X - np.mean(X, axis=0), eigv_f)

In [85]:
X_trans

array([[ 0.46614617, -0.24694603],
       [-0.00805462,  0.07397737],
       [ 0.27668192,  0.17386488],
       [ 0.02792672,  0.38758932],
       [-0.21516258,  0.42307254],
       [-0.43788782, -0.02701655],
       [ 0.41228367, -0.45558603],
       [-0.61135748, -0.12266951],
       [-0.34381764, -0.43521266],
       [ 0.43324165,  0.22892666]])

In [86]:
np.cov(X_trans, rowvar=False)

array([[1.53597254e-01, 6.81601423e-18],
       [6.81601423e-18, 9.90059616e-02]])

In [89]:
np.dot(X_trans, eigv_f.T) + np.mean(X, axis=0)

array([[ 0.82454907,  0.26436313,  0.15336152],
       [ 0.65885069,  0.59345345,  0.59165573],
       [ 0.90522998,  0.41960538,  0.58040831],
       [ 0.8410875 ,  0.59592299,  0.84939739],
       [ 0.69136927,  0.75414794,  0.96297901],
       [ 0.31191871,  0.85993437,  0.65150954],
       [ 0.68268693,  0.28184337, -0.00788797],
       [ 0.1443478 ,  0.9630222 ,  0.62839891],
       [ 0.1718273 ,  0.76667705,  0.26790723],
       [ 1.04076939,  0.32402797,  0.57434418]])

In [76]:
np.dot(eigv_f.T, eigv_f)

array([[ 1.00000000e+00, -6.24504298e-17],
       [-6.24504298e-17,  1.00000000e+00]])